In [22]:
import pandas as pd

df = pd.read_csv('StudentsPerformance.csv')
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [23]:
df = df.drop(columns=['race/ethnicity'])
df

,gender,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,bachelor's degree,standard,none,72,72,74
1,female,some college,standard,completed,69,90,88
2,female,master's degree,standard,none,90,95,93
3,male,associate's degree,free/reduced,none,47,57,44
4,male,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...
995,female,master's degree,standard,completed,88,99,95
996,male,high school,free/reduced,none,62,55,55
997,female,high school,free/reduced,completed,59,71,65
998,female,some college,standard,completed,68,78,77


In [24]:
# Group A - White - British
# Group B - White - Irish
# Group C - White - Any other White background
# Group D - Mixed - White and Black Caribbean
# Group E - Mixed - White and Black African
# Group F - Mixed - White and Asian
# Group G - Mixed - Any other mixed background
df_one = pd.get_dummies(df["gender"])# f
df_one


,female,male
0,1,0
1,1,0
2,1,0
3,0,1
4,0,1
...,...,...
995,1,0
996,0,1
997,1,0
998,1,0


In [ ]:
from sklearn import preprocessing
en = preprocessing.OrdinalEncoder()
en.fit(df)

In [ ]:
import seaborn as sns

_ = sns.pairplot(df)